In [1]:
import numpy as np
import pygame
from queue import Queue
from queue import PriorityQueue

pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def read_maze(filename="Maze.txt"):
    with open(filename, 'r') as file:
        maze_str=file.read()     
    return maze_str

In [3]:
def Maze_Convert_to_Array_Finding_Start_Goal(maze_str):
    temp_rows=maze_str.split('\n')
    temp_lst=[]
    
    for i,x in enumerate(temp_rows):
        lst=[]
        for j,y in enumerate(x.split()):
            if y=="S":
                Start=(i,j)
                lst.append(0)
            elif y=="G":
                Goal=(i,j)
                lst.append(0)
            else:
                lst.append(int(y))
        temp_lst.append(lst)    
    array_maze=np.array(temp_lst)
    return array_maze,Start,Goal

In [4]:
def get_neighbors(x, y, maze):
    #create a list of neighbors
    neighbours = []
    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        nx, ny = x + dx, y + dy
        # boundry check
        if (0<=nx<len(maze)) and (0<=ny<len(maze[nx])):
            #wall check
            if maze[nx][ny]!=1:
                neighbours.append((nx,ny))
    return neighbours

In [5]:
def BFS_Find_Path(maze,start,goal):
    #visited unique list
    visited=set()
    #create a queue
    que=Queue()
    #putting xpos ypos and path
    x,y=start
    que.put( (x,y,[start]) )
    #queue is not empty
    while not que.empty():
        x,y,path = que.get()
        if (x,y)==goal:
            return path
        if (x,y) not in visited:
            visited.add((x,y))
            neighbours=get_neighbors(x,y,maze)
            for n in neighbours:
                x,y=n
                que.put( (x,y,path+[(x,y)]) )
                
    return -1

In [6]:
def DFS_Find_Path(maze,start,goal):
    visited = set()
    x,y=start
    stack = [(x,y, [start])]
    while stack:
        x,y,path =stack.pop()
        if (x,y)==goal:
            return path
        if (x,y) not in visited:
            visited.add((x,y))
            neighbours=get_neighbors(x,y,maze)
            for n in neighbours:
                x,y=n
                stack.append( (x,y,path+[(x,y)]) ) 
        
    return -1

In [7]:
def Calculate_H(maze,cell1,cell2):
    #calculationg manhatting distance btw current cell and goal and if cell is short jump(2) then 500 cost is added
    #Manhattan Distance = | x 1 − x 2 | + | y 1 − y 2 | 
    x1,y1=cell1
    x2,y2=cell2
    h=abs(x1-x2)+abs(y1-y2)
    if maze[x1][y1]==2:
        h+=500
    return h

In [8]:
def Sorted_Unique_Path(fwd_path):
    path=set()
    for i,j in fwd_path.items():
        path.add(i)
        path.add(j)
    return sorted(path)

In [9]:
def Find_A_Star_Path(maze_array,start,goal):
    gscore={}
    fscore={}
    for i,x in enumerate(maze_array):
        for j,y in enumerate(x):
            gscore[(i,j)]=float('inf')
            fscore[(i,j)]=float('inf')

    que=PriorityQueue()
    x,y=start
    # path can ce be stored as dict path[child_cell]=currcell
    # cant be stored as path[currcell]=child_cell as it can only one value other values might get deleted
    gscore[start]=0
    fscore[start]=Calculate_H(maze_array,start,goal)
    que.put((fscore[start],Calculate_H(maze_array,start,goal),start))
    apath={}
    while not que.empty():
        temp_fscore,temp_hscore,current_cell=que.get()
        x,y=current_cell
        neighbours=get_neighbors(x,y,maze_array)
        for i,child in enumerate(neighbours):
            temp_gscore=gscore[current_cell]+1 if maze_array[x][y]!=2 else gscore[current_cell]+5
            temp_fscore=temp_gscore+Calculate_H(maze_array,child,goal)
            if temp_fscore<fscore[child]:

                fscore[child]=temp_fscore
                gscore[child]=temp_gscore
                apath[child]=current_cell
                que.put( (fscore[child],Calculate_H(maze_array,child,goal),child   ) )

    fwd_path=dict()
    temp_cell=goal
    if goal in apath.keys() :
    #check for path found to goal
        while temp_cell!=start:

            fwd_path[apath[temp_cell]]=temp_cell
            temp_cell=apath[temp_cell]

        return Sorted_Unique_Path(fwd_path)
    else :
        return -1

In [10]:
def Visualize_Maze(maze, path):
    # Define some colors
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    GREEN = (0, 255, 0)
    RED = (255, 0, 0)
    Blue=(0,0,255)
    Orange=( 255 , 100 , 10)
    Brown = ( 100 , 40 , 0 )
    # Initialize pygame
    pygame.init()
    
    # Set the dimensions of the window
    WINDOW_SIZE = [700, 700]
    screen = pygame.display.set_mode(WINDOW_SIZE)
    
    # Set the title of the window
    pygame.display.set_caption("Maze Solver")
    
    # Set the font for the text
    font = pygame.font.SysFont('arial', 20)
    
    # Calculate the size of each cell in the maze
    cell_size = min(WINDOW_SIZE[0] // len(maze[0]), WINDOW_SIZE[1] // len(maze))
    
    # Define the starting and ending points
    start = (0, 0)
    end = (len(maze[0]) - 1, len(maze) - 1)
    
    # Draw the maze
    for row in range(len(maze)):
        for col in range(len(maze[0])):
            color = WHITE
            if maze[row][col] == 1:
                color = BLACK
            elif maze[row][col] == 2:
                color = Brown
            pygame.draw.rect(screen, color, [col * cell_size, row * cell_size, cell_size, cell_size])
    
    # Draw the starting and ending points
    pygame.draw.rect(screen, GREEN, [start[1] * cell_size, start[0] * cell_size, cell_size, cell_size])
    pygame.draw.rect(screen, RED, [end[1] * cell_size, end[0] * cell_size, cell_size, cell_size])
    
    # Draw the path
    if path != -1:
        for i in range(len(path) - 1):
            pygame.draw.line(screen, GREEN, [path[i][1] * cell_size + cell_size // 2, path[i][0] * cell_size + cell_size // 2], [path[i+1][1] * cell_size + cell_size // 2, path[i+1][0] * cell_size + cell_size // 2], 5)
    
    # Add text to the screen
    if path != -1:
        text = font.render("Path found!", True, Blue)
    else:
        text = font.render("No path found.", True, Orange)
        
    #Screen postion to display text
    screen.blit(text, [300, 300])
    
    # Update the screen
    pygame.display.flip()
    
    # Wait for the user to close the window
    done = False
    while not done:
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
    
    # Quit pygame
    pygame.quit()


In [11]:
def Start_Maze_Search(option):
    maze_str=read_maze("Maze4.txt")
    maze_array,start,goal=Maze_Convert_to_Array_Finding_Start_Goal(maze_str)
    if option==1:
        path_bfs=BFS_Find_Path(maze_array,start,goal)
        Visualize_Maze(maze_array,path_bfs)
    elif option==2:
        path_dfs=DFS_Find_Path(maze_array,start,goal)
        Visualize_Maze(maze_array,path_dfs) 
    elif option==3:
        maze_str=read_maze("Maze5.txt")
        maze_array,start,goal=Maze_Convert_to_Array_Finding_Start_Goal(maze_str)
        path_A_star=Find_A_Star_Path(maze_array,start,goal)
        Visualize_Maze(maze_array,path_A_star)

In [12]:
print("1. Breadth First Search : ")
print("2. Depth First Search : ")
print("3. A* Search : ")
choice = int(input("Enter your choice : "))
if choice==1:
    print("Now Showing path in maze using BFS :")
    Start_Maze_Search(choice)
elif choice==2:
    print("Now Showing path in maze using DFS :")
    Start_Maze_Search(choice)
elif choice==3:
    print("Now Showing path in maze using A * :")
    Start_Maze_Search(choice)

1. Breadth First Search : 
2. Depth First Search : 
3. A* Search : 
Enter your choice : 3
Now Showing path in maze using A * :
